In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
from edward.models import Normal
from edward.models import Beta
from edward.models import Bernoulli, MultivariateNormalTriL, Categorical

import numpy as np
import tensorflow as tf
from getEmbeddings import getEmbeddings
import matplotlib.pyplot as plt
import scikitplot.plotters as skplt
import os.path

In [12]:
IN_DIM = 300
CLASS_NUM = 2
LEARN_RATE = 0.0001
TRAIN_STEP = 20000
tensorflow_tmp = "tmp_tensorflow"

In [13]:
N = 16608
D = IN_DIM

In [14]:
tf.reset_default_graph()

In [15]:
def data():
    # Get the training and testing data from getEmbeddings
    # Read the Doc2Vec data
    train_data = np.load('../Data/xtr.npy')
    eval_data = np.load('../Data/xte.npy')
    train_labels = np.load('../Data/ytr.npy')
    eval_labels = np.load('../Data/yte.npy')
    train_labels = train_labels.reshape((-1, 1)).astype(np.int32)

    return train_data, train_labels.flatten(), eval_data, eval_labels.flatten()

In [16]:
'''def neural_network(X):
    h = tf.nn.relu(tf.matmul(X, W_0) + b_0)
    h = tf.nn.relu(tf.matmul(h, W_1) + b_1)
    h = tf.nn.relu(tf.matmul(h, W_2) + b_2)
    h = tf.nn.relu(tf.matmul(h, W_3) + b_3)
    return tf.reshape(h, [-1])

ed.set_seed(42)
'''
def neural_network(X):
    h = tf.tanh(tf.matmul(X, W_0) + b_0)
    h = tf.tanh(tf.matmul(h, W_1) + b_1)
    h = tf.tanh(tf.matmul(h, W_2) + b_2)
    h = tf.tanh(tf.matmul(h, W_3) + b_3)
    return h#tf.reshape(h, [-1])

ed.set_seed(42)

In [17]:
'''
with tf.name_scope("model"):
    W_0 = Normal(loc=tf.zeros([D, 300]), scale=tf.ones([D, 300]),
                 name="W_0")
    W_1 = Normal(loc=tf.zeros([300, 300]), scale=tf.ones([300, 300]), name="W_1")
    W_2 = Normal(loc=tf.zeros([300, 300]), scale=tf.ones([300, 300]), name="W_2")
    W_3 = Normal(loc=tf.zeros([300, 2]), scale=tf.ones([300, 2]), name="W_3")
    b_0 = Normal(loc=tf.zeros(300), scale=tf.ones(300), name="b_0")
    b_1 = Normal(loc=tf.zeros(300), scale=tf.ones(300), name="b_1")
    b_2 = Normal(loc=tf.zeros(300), scale=tf.ones(300), name="b_2")
    b_3 = Normal(loc=tf.ones(2), scale=tf.ones(2), name="b_3")  
    X = tf.placeholder(tf.float32, [None, D], name="X")
    y = Categorical(neural_network(X), name="out")
'''

'\nwith tf.name_scope("model"):\n    W_0 = Normal(loc=tf.zeros([D, 300]), scale=tf.ones([D, 300]),\n                 name="W_0")\n    W_1 = Normal(loc=tf.zeros([300, 300]), scale=tf.ones([300, 300]), name="W_1")\n    W_2 = Normal(loc=tf.zeros([300, 300]), scale=tf.ones([300, 300]), name="W_2")\n    W_3 = Normal(loc=tf.zeros([300, 2]), scale=tf.ones([300, 2]), name="W_3")\n    b_0 = Normal(loc=tf.zeros(300), scale=tf.ones(300), name="b_0")\n    b_1 = Normal(loc=tf.zeros(300), scale=tf.ones(300), name="b_1")\n    b_2 = Normal(loc=tf.zeros(300), scale=tf.ones(300), name="b_2")\n    b_3 = Normal(loc=tf.ones(2), scale=tf.ones(2), name="b_3")  \n    X = tf.placeholder(tf.float32, [None, D], name="X")\n    y = Categorical(neural_network(X), name="out")\n'

In [18]:
from keras.utils.np_utils import to_categorical
from edward.models import Normal, OneHotCategorical

X_train, y_train, X_test, y_test = data()
#y_train = to_categorical(y_train, num_classes=2)

In [19]:
X_train[0]

array([-0.10909925, -0.1533924 ,  0.25079963, -0.44075423,  0.76589823,
        0.06528959,  0.33648187, -0.39045393,  0.19403008, -0.15183719,
       -0.31247854,  0.01048287, -0.00943268,  0.39518169, -0.42421004,
       -0.02285684,  0.25530708,  0.21203719,  0.25060442, -0.11962883,
        0.18786967, -0.12573212,  0.63139296,  0.51808155,  0.31717432,
       -0.26647913,  0.64298368,  0.05415455, -0.34272864,  0.20884421,
       -0.0406697 , -0.18859312,  0.03097081, -0.02661392, -0.26044583,
       -0.11376627, -0.55710369,  0.41282558,  0.16852812, -0.31658146,
        0.40650964,  0.19828998,  0.06955437,  0.48007151,  0.23204562,
       -0.07754508,  0.23100746, -0.27005479,  0.11164568, -0.06827326,
       -0.04374921, -0.1149475 , -0.18935089, -0.27474064, -0.18915153,
       -0.1176029 , -0.39521331,  0.0922235 , -0.11931918,  0.20645793,
        0.37922385,  0.48577979, -0.20960642,  0.48669472,  0.41080502,
        0.04662562,  0.29828101,  0.03053242, -0.61243594,  0.13

In [20]:
'''
# INFERENCE
with tf.variable_scope("posterior"):
    with tf.variable_scope("qW_0"):
      loc = tf.get_variable("loc", [D, 300])
      scale = tf.nn.softplus(tf.get_variable("scale", [D, 300]))
      qW_0 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_1"):
      loc = tf.get_variable("loc", [300, 300])
      scale = tf.nn.softplus(tf.get_variable("scale", [300, 300]))
      qW_1 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_2"):
      loc = tf.get_variable("loc", [300, 300])
      scale = tf.nn.softplus(tf.get_variable("scale", [300, 300]))
      qW_2 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_3"):
      loc = tf.get_variable("loc", [300, 2])
      scale = tf.nn.softplus(tf.get_variable("scale", [300, 2]))
      qW_3 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_0"):
      loc = tf.get_variable("loc", [300])
      scale = tf.nn.softplus(tf.get_variable("scale", [300]))
      qb_0 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_1"):
      loc = tf.get_variable("loc", [300])
      scale = tf.nn.softplus(tf.get_variable("scale", [300]))
      qb_1 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_2"):
      loc = tf.get_variable("loc", [300])
      scale = tf.nn.softplus(tf.get_variable("scale", [300]))
      qb_2 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_3"):
      loc = tf.get_variable("loc", [2])
      scale = tf.nn.softplus(tf.get_variable("scale", [2]))
      qb_3 = Normal(loc=loc, scale=scale)
'''

'\n# INFERENCE\nwith tf.variable_scope("posterior"):\n    with tf.variable_scope("qW_0"):\n      loc = tf.get_variable("loc", [D, 300])\n      scale = tf.nn.softplus(tf.get_variable("scale", [D, 300]))\n      qW_0 = Normal(loc=loc, scale=scale)\n    with tf.variable_scope("qW_1"):\n      loc = tf.get_variable("loc", [300, 300])\n      scale = tf.nn.softplus(tf.get_variable("scale", [300, 300]))\n      qW_1 = Normal(loc=loc, scale=scale)\n    with tf.variable_scope("qW_2"):\n      loc = tf.get_variable("loc", [300, 300])\n      scale = tf.nn.softplus(tf.get_variable("scale", [300, 300]))\n      qW_2 = Normal(loc=loc, scale=scale)\n    with tf.variable_scope("qW_3"):\n      loc = tf.get_variable("loc", [300, 2])\n      scale = tf.nn.softplus(tf.get_variable("scale", [300, 2]))\n      qW_3 = Normal(loc=loc, scale=scale)\n    with tf.variable_scope("qb_0"):\n      loc = tf.get_variable("loc", [300])\n      scale = tf.nn.softplus(tf.get_variable("scale", [300]))\n      qb_0 = Normal(loc=loc

In [21]:
with tf.name_scope("model"):
    W_0 = Normal(loc=tf.zeros([D, 256]), scale=tf.ones([D, 256]),
                 name="W_0")
    W_1 = Normal(loc=tf.zeros([256, 256]), scale=tf.ones([256, 256]), name="W_1")
    W_2 = Normal(loc=tf.zeros([256, 80]), scale=tf.ones([256, 80]), name="W_2")
    W_3 = Normal(loc=tf.zeros([80, 2]), scale=tf.ones([80, 2]), name="W_3")
    b_0 = Normal(loc=tf.zeros(256), scale=tf.ones(256), name="b_0")
    b_1 = Normal(loc=tf.zeros(256), scale=tf.ones(256), name="b_1")
    b_2 = Normal(loc=tf.zeros(80), scale=tf.ones(80), name="b_2")
    b_3 = Normal(loc=tf.ones(2), scale=tf.ones(2), name="b_3")  
    X = tf.placeholder(tf.float32, [None, D], name="X")
    y = Categorical(neural_network(X), name="out")
    
# INFERENCE
with tf.variable_scope("posterior"):
    with tf.variable_scope("qW_0"):
      loc = tf.get_variable("loc", [D, 256])
      scale = tf.nn.softplus(tf.get_variable("scale", [D, 256]))
      qW_0 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_1"):
      loc = tf.get_variable("loc", [256, 256])
      scale = tf.nn.softplus(tf.get_variable("scale", [256, 256]))
      qW_1 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_2"):
      loc = tf.get_variable("loc", [256, 80])
      scale = tf.nn.softplus(tf.get_variable("scale", [256, 80]))
      qW_2 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_3"):
      loc = tf.get_variable("loc", [80, 2])
      scale = tf.nn.softplus(tf.get_variable("scale", [80, 2]))
      qW_3 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_0"):
      loc = tf.get_variable("loc", [256])
      scale = tf.nn.softplus(tf.get_variable("scale", [256]))
      qb_0 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_1"):
      loc = tf.get_variable("loc", [256])
      scale = tf.nn.softplus(tf.get_variable("scale", [256]))
      qb_1 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_2"):
      loc = tf.get_variable("loc", [80])
      scale = tf.nn.softplus(tf.get_variable("scale", [80]))
      qb_2 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_3"):
      loc = tf.get_variable("loc", [2])
      scale = tf.nn.softplus(tf.get_variable("scale", [2]))
      qb_3 = Normal(loc=loc, scale=scale)

In [22]:
'''
zero_initializer = tf.constant_initializer(value=0.0)
ones_initializer = tf.constant_initializer(value=1.0)
with tf.variable_scope("posterior"):
    with tf.variable_scope("qW_0"):
      loc = tf.get_variable("loc", [D, 1000], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [D, 1000], initializer=ones_initializer))
      qW_0 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_1"):
      loc = tf.get_variable("loc", [1000, 500], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [1000, 500], initializer=ones_initializer))
      qW_1 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_2"):
      loc = tf.get_variable("loc", [500, 300], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [500, 300], initializer=ones_initializer))
      qW_2 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_3"):
      loc = tf.get_variable("loc", [300, 1], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [300, 1], initializer=ones_initializer))
      qW_3 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_0"):
      loc = tf.get_variable("loc", [1000], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [1000], initializer=ones_initializer))
      qb_0 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_1"):
      loc = tf.get_variable("loc", [500], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [500], initializer=ones_initializer))
      qb_1 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_2"):
      loc = tf.get_variable("loc", [300], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [300], initializer=ones_initializer))
      qb_2 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_3"):
      loc = tf.get_variable("loc", [1], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [1], initializer=ones_initializer))
      qb_3 = Normal(loc=loc, scale=scale)
'''

'\nzero_initializer = tf.constant_initializer(value=0.0)\nones_initializer = tf.constant_initializer(value=1.0)\nwith tf.variable_scope("posterior"):\n    with tf.variable_scope("qW_0"):\n      loc = tf.get_variable("loc", [D, 1000], initializer=zero_initializer)\n      scale = tf.nn.softplus(tf.get_variable("scale", [D, 1000], initializer=ones_initializer))\n      qW_0 = Normal(loc=loc, scale=scale)\n    with tf.variable_scope("qW_1"):\n      loc = tf.get_variable("loc", [1000, 500], initializer=zero_initializer)\n      scale = tf.nn.softplus(tf.get_variable("scale", [1000, 500], initializer=ones_initializer))\n      qW_1 = Normal(loc=loc, scale=scale)\n    with tf.variable_scope("qW_2"):\n      loc = tf.get_variable("loc", [500, 300], initializer=zero_initializer)\n      scale = tf.nn.softplus(tf.get_variable("scale", [500, 300], initializer=ones_initializer))\n      qW_2 = Normal(loc=loc, scale=scale)\n    with tf.variable_scope("qW_3"):\n      loc = tf.get_variable("loc", [300, 1],

In [32]:
#lets do inference
inference = ed.KLqp({W_0: qW_0, b_0: qb_0,
                   W_1: qW_1, b_1: qb_1,
                   W_2: qW_2, b_2: qb_2,
                   W_3: qW_3, b_3: qb_3}, data={X: X_train, y: y_train})
inference.run(logdir='log', n_iter=100)

100/100 [100%] ██████████████████████████████ Elapsed: 22s | Loss: 28360.969


In [33]:
ed.evaluate('categorical_accuracy', data={y: y_train, X: X_train})

0.5089114

In [34]:
ed.evaluate('categorical_accuracy', data={y: y_test, X: X_test})

0.5916205